In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import glob
import os
import pandas as pd

import efficientnet.tfkeras as efn

In [2]:
train_image_path = glob.glob('训练集\\train\\*\\*.png')
labels = [int(i.split('\\')[2]) for i in train_image_path]

In [3]:
# Step1 使用 tf.data.Dataset.from_tensor_slices 进行加载
image_ds = tf.data.Dataset.from_tensor_slices((train_image_path,labels))

In [4]:
val_count = int(len(labels)*0.2)
train_count = len(labels)-val_count

In [5]:
def load_preprocess_image(path,label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image,channels=3)
    image = tf.image.resize(image,[300,300])
    image = tf.image.random_crop(image,[260,260,3])
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image,0.2)
    
    image = tf.cast(image,tf.float32)
    image = image/255
    label = tf.reshape(label,[1])
    return image,label

In [6]:
def load_preprocess_image_test(path,label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image,channels=3)
    image = tf.image.resize(image,[260,260])
    image = tf.cast(image,tf.float32)
    image = image/255
    label = tf.reshape(label,[1])
    return image,label

In [7]:
# 选取验证集
image_train_ds = image_ds.skip(val_count)
image_val_ds = image_ds.take(val_count)

# Step3 预处理 (预处理函数在下面)
AUTOTUNE = tf.data.experimental.AUTOTUNE
image_train_ds = image_train_ds.map(load_preprocess_image,num_parallel_calls=AUTOTUNE)
image_val_ds = image_val_ds.map(load_preprocess_image_test,num_parallel_calls=AUTOTUNE)

In [8]:
BATCH_SIZE = 32
# shuffle 打乱数据 batch设置 batch size repeat设置迭代次数(迭代2次) test数据集不需要
image_train_ds = image_train_ds.repeat(2).shuffle(train_count).batch(BATCH_SIZE)
image_train_ds = image_train_ds.prefetch(AUTOTUNE)#预取,GPU，CPU加速

image_val_ds = image_val_ds.batch(BATCH_SIZE)
image_val_ds = image_val_ds.prefetch(AUTOTUNE)#预取

In [9]:
covn_base = efn.EfficientNetB3(weights='imagenet',
                               input_shape=(260,260,3),
                               include_top=False,
                               pooling='avg')

In [1]:
# covn_base.summary()

In [13]:
model = keras.Sequential()
model.add(covn_base)
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1024,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(512,activation='relu'))
model.add(keras.layers.Dense(24,activation='sigmoid'))

covn_base.trainable = False #设置权重参数不可训练

In [14]:
#编译
model.compile(optimizer=keras.optimizers.Adam(lr=0.001),
             loss = 'sparse_categorical_crossentropy',
             metrics=['acc'])

history = model.fit(
    image_train_ds,
    steps_per_epoch=train_count//BATCH_SIZE,
    epochs=8,
    validation_data=image_val_ds,
    validation_steps=val_count//BATCH_SIZE
)

Epoch 1/8
46/46 [==============================] - 200s 4s/step - loss: 2.9939 - acc: 0.0768 - val_loss: 11.0184 - val_acc: 0.0000e+00
Epoch 2/8
46/46 [==============================] - 229s 5s/step - loss: 2.9694 - acc: 0.0808 - val_loss: 12.8389 - val_acc: 0.0000e+00
Epoch 3/8
 2/46 [>.............................] - 41s 21s/step - loss: 2.9774 - acc: 0.0000e+00 - val_loss: 12.8920 - val_acc: 0.0000e+00


In [16]:
covn_base.trainable=True
fine_tune_at = -3
for layer in covn_base.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(optimizer=keras.optimizers.Adam(0.0005),
             loss = 'sparse_categorical_crossentropy',
             metrics=['acc'])

initial_epochs=8
fine_tune_epochs=8
total_epoch = initial_epochs+fine_tune_epochs

history = model.fit(
    image_train_ds,
    steps_per_epoch=train_count//BATCH_SIZE,
    epochs=total_epoch,
    initial_epoch=initial_epochs,
    validation_data=image_val_ds,
    validation_steps=val_count//BATCH_SIZE
)

model.save('mstz_model_EfficientNetB52.h5')

Epoch 9/16
46/46 [==============================] - 231s 5s/step - loss: 2.8240 - acc: 0.1345 - val_loss: 12.0627 - val_acc: 0.0000e+00
Epoch 10/16
46/46 [==============================] - 215s 5s/step - loss: 2.6769 - acc: 0.1773 - val_loss: 11.6352 - val_acc: 0.0000e+00
Epoch 11/16
 2/46 [>.............................] - 49s 24s/step - loss: 2.4950 - acc: 0.0682 - val_loss: 11.6590 - val_acc: 0.0000e+00


In [17]:
#加载图片
def load_preprocess_images(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image,channels=3)
    image = tf.image.resize(image,[260,260])
    image = tf.cast(image,tf.float32)
    image = image/255
    image = tf.expand_dims(image,0)
    return image

In [18]:
test_image_path = glob.glob('测试集/*.png')

test_image_path.sort(key=lambda x:int(x.split('\\')[-1].split('.')[0][1:]))

images = [load_preprocess_images(image_path) for image_path in test_image_path]

In [20]:
image_count = len(images)
values = []
result_dict = {}

for i in range(image_count):
    pred = model.predict(images[i])
    values.append(np.argmax(pred))
    print('.',end='')

..........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [21]:
#写文件
with open('result_efficientB2.csv','w',encoding='utf-8') as f:
    f.write('image_id,category_id\n')
    [f.write('{0},{1}\n'.format(key.split('\\')[1][:-4]+'.png', value)) for (key,value) in zip(test_image_path,values)]